# Create Nodes

To run this experiment, you should have already created an account on Chameleon, become part of a project, reserve three `compute_gigaio` nodes(`compute_gigaio_project46_1`, `compute_gigaio_project46_2`,`compute_gigaio_project46_3`). You should also have added your SSH key to the CHI@UC site.

In [1]:
# runs in Chameleon Jupyter environment
from chi import context
import os

context.version = "1.0" 
context.choose_project()
context.choose_site(default="CHI@UC")

In [2]:
from chi import lease

lease_names = {
    "node1": "compute_gigaio_project46_1",
    "node2": "compute_gigaio_project46_2",
    "node3": "compute_gigaio_project46_3"
}

reservation_map = {
    name: lease.get_lease(lease_name).node_reservations[0]["id"]
    for name, lease_name in lease_names.items()
}
# node_hostnames = {
#     name: lease.get_lease(lease_name).nodes[0]["name"] + ".chameleoncloud.org"
#     for name, lease_name in lease_names.items()
# }
for name, lease_name in lease_names.items():
    print(lease.get_lease(lease_name).node_reservations[0]["id"])

a09e00c8-af62-4b92-929b-1e39fb4020cd
ad5247ff-9e60-49ff-a862-3d52a6bf09e6
eca3a12c-8edb-455e-b873-b18fa1ee39dd


In [ ]:
cd ~/Fine-Tuning-Taiwanese-Hokkien-LLM-for-Medical-Advising/iac/tf/chi 

In [4]:
import json

with open("outputs.json") as f:
    out = json.load(f)
    sharednet1_ports = {
        "node1": out["sharednet1_ports"]["value"]["node1"],
        "node2": out["sharednet1_ports"]["value"]["node2"],
        "node3": out["sharednet1_ports"]["value"]["node3"],
    }
    private_net_ports = {
        "node1": out["private_net_ports"]["value"]["node1"],
        "node2": out["private_net_ports"]["value"]["node2"],
        "node3": out["private_net_ports"]["value"]["node3"],
    }

In [5]:
# path = os.path.expanduser("~/Fine-Tuning-Taiwanese-Hokkien-LLM-for-Medical-Advising/iac/tf/chi/terraform.tfvars")
# with open(path, "w") as f:
#     f.write('suffix = "yc7690"\n')
#     f.write('key    = "id_rsa_chameleon"\n')
#     f.write('node_reservations = {\n')
#     for node, res_id in reservation_map.items():
#         f.write(f'  {node} = "{res_id}"\n')
#     f.write('}\n')
# with open(path, "w") as f:
#     f.write("node_hostnames = {\n")
#     for name, host in node_hostnames.items():
#         f.write(f'  {name} = "{host}"\n')
#     f.write("}\n")

In [6]:
# from openstack import connection
# import subprocess, json

# terraform_dir = os.path.expanduser("~/Fine-Tuning-Taiwanese-Hokkien-LLM-for-Medical-Advising/iac/tf/chi")
# outputs = subprocess.check_output(["terraform", "output", "-json"], cwd=terraform_dir)

# ports = json.loads(outputs)
# port_map = {
#     "node1": ports["sharednet1_port_id_node1"]["value"],
#     "node2": ports["private_net_port_id_node2"]["value"],
#     "node3": ports["private_net_port_id_node3"]["value"]
# }
# print(ports.keys())

In [7]:
# Create servers
# from openstack import connection
# conn = connection.from_config(cloud="openstack")
# for node in ["node1", "node2", "node3"]:
#     print(f"Creating {node}...")
#     server = conn.create_server(
#         name=f"{node}-mlops",
#         image='CC-Ubuntu24.04-CUDA',
#         flavor='baremetal',
#         key_name='id_rsa_chameleon',
#         nics=[
#             {"port-id": sharednet1_ports[node]},
#             {"port-id": private_net_ports[node]}
#         ],
#         scheduler_hints={"reservation": reservation_map[node]},
#         wait=False,
#         auto_ip=False,
#         user_data = f"""#!/bin/bash
# echo '127.0.1.1 {node}-mlops' >> /etc/hosts
# su cc -c /usr/local/bin/cc-load-public-keys
# """
#     )
#     print(f"{node} server created: {server.id}")


In [8]:
import base64

user_data_script = """#!/bin/bash
echo '127.0.1.1 {node}-mlops' >> /etc/hosts
su cc -c /usr/local/bin/cc-load-public-keys
"""
from openstack import connection
conn = connection.from_config(cloud="openstack")
for node in ["node1", "node2", "node3"]:
    print(f"Creating {node}...")

    encoded_user_data = base64.b64encode(
        user_data_script.format(node=node).encode("utf-8")
    ).decode("utf-8")

    server = conn.compute.create_server(
        name=f"{node}-mlops",
        image_id=conn.compute.find_image("CC-Ubuntu24.04-CUDA").id,
        flavor_id=conn.compute.find_flavor("baremetal").id,
        networks=[
            {"port": sharednet1_ports[node]},
            {"port": private_net_ports[node]}
        ],
        key_name="id_rsa_chameleon",
        scheduler_hints={"reservation": reservation_map[node]},
        user_data=encoded_user_data
    )
    print(f"{node} server created: {server.id}")


Creating node1...
node1 server created: 0e34eabd-fffd-450b-af60-a244b6a13614
Creating node2...
node2 server created: 5f228da3-7d3c-4f27-af04-510cae12d3c7
Creating node3...
node3 server created: 90278510-f28e-418b-8b74-98b7824777b9


In [13]:
server = conn.get_server("90278510-f28e-418b-8b74-98b7824777b9")
print(server.status)

ACTIVE
